In [2]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [7]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [8]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


(91795, 8)


In [9]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.LSTM(64,return_sequences=True,dropout=0.3),
    tf.keras.layers.LSTM(32,return_sequences=True,dropout=0.3),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           8000032   
_________________________________________________________________
lstm (LSTM)                  (None, 200, 128)          82432     
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 64)           49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 32)           12416     
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
Total params: 8,147,560
Trainable params: 8,147,560
Non-trainable params: 0
______________________________________________

In [10]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


Epoch 1/30
323/323 [==============================] - 31s 96ms/step - loss: 1.9556 - acc: 0.2211 - val_loss: 1.9177 - val_acc: 0.2407
Epoch 2/30
323/323 [==============================] - 30s 93ms/step - loss: 1.9459 - acc: 0.2196 - val_loss: 1.9414 - val_acc: 0.2182
Epoch 3/30
323/323 [==============================] - 30s 93ms/step - loss: 1.9446 - acc: 0.2126 - val_loss: 1.9600 - val_acc: 0.2078
Epoch 4/30
323/323 [==============================] - 30s 92ms/step - loss: 1.9939 - acc: 0.2020 - val_loss: 1.9356 - val_acc: 0.2439
Epoch 5/30
323/323 [==============================] - 30s 92ms/step - loss: 1.9336 - acc: 0.2413 - val_loss: 1.9365 - val_acc: 0.2429
Epoch 6/30
323/323 [==============================] - 30s 92ms/step - loss: 1.9278 - acc: 0.2437 - val_loss: 1.9077 - val_acc: 0.2526
Epoch 7/30
323/323 [==============================] - 30s 92ms/step - loss: 1.9313 - acc: 0.2320 - val_loss: 2.0556 - val_acc: 0.1661
Epoch 8/30
323/323 [==============================] - 30s 92ms

In [11]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 6400)
(75271, 8)
(16524, 1, 6400)
(16524, 8)


In [14]:
model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(2048,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.LSTM(1024,return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.LSTM(512,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.LSTM(64,return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(8, activation="softmax")
])

In [15]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 2048)           69214208  
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 1024)           12587008  
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 512)            3147776   
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 256)            787456    
_________________________________________________________________
lstm_8 (LSTM)                (None, 1, 64)             82176     
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                

In [16]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

Epoch 1/25
265/265 [==============================] - 13s 51ms/step - loss: 1.0585 - acc: 0.5288 - val_loss: 0.4760 - val_acc: 0.8571
Epoch 2/25
265/265 [==============================] - 12s 45ms/step - loss: 0.1862 - acc: 0.9671 - val_loss: 0.0972 - val_acc: 0.9810
Epoch 3/25
265/265 [==============================] - 12s 45ms/step - loss: 0.1089 - acc: 0.9776 - val_loss: 0.0824 - val_acc: 0.9822
Epoch 4/25
265/265 [==============================] - 12s 45ms/step - loss: 0.0973 - acc: 0.9789 - val_loss: 0.0916 - val_acc: 0.9798
Epoch 5/25
265/265 [==============================] - 12s 45ms/step - loss: 0.0918 - acc: 0.9799 - val_loss: 0.0848 - val_acc: 0.9805
Epoch 6/25
265/265 [==============================] - 12s 45ms/step - loss: 0.0876 - acc: 0.9806 - val_loss: 0.0791 - val_acc: 0.9830
Epoch 7/25
265/265 [==============================] - 12s 45ms/step - loss: 0.0862 - acc: 0.9815 - val_loss: 0.0775 - val_acc: 0.9829
Epoch 8/25
265/265 [==============================] - 12s 45ms

In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_word_embedding')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [17]:
results = model.evaluate(X_test, y_test)
print(results)

517/517 [==============================] - 4s 8ms/step - loss: 0.0674 - acc: 0.9851
[0.06736298650503159, 0.9850520491600037]


In [0]:
y_pred=model.predict(X_test)

In [19]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [20]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

98.47554828970641
98.4314804558961
98.4524553144526
